## Segmenting and clustering neighborhoods - Toronto
## Task 2 - getting geospatial data
### See section "Geospatial data" for the code for getting location data

The first few cells are a repetition from task 1.

Importing the libraries including BeautifulSoup for web scraping tasks:

In [1]:
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup

# use module dotenv to manage API keys and secrets
import os
%load_ext dotenv
%dotenv

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.geocoders import ArcGIS # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Data gathering
Collect data from wikipedia with beautiful soup:

In [2]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(wiki_url).text
soup = BeautifulSoup(website_url,'html.parser')
table = soup.find('table', attrs={'class':'wikitable sortable'})
rows = table.find_all('tr')[1:]
list_codes = []
for row in rows:
    cols = row.find_all('td')
    list_codes.append({'PostalCode':cols[0].text, 'Borough':cols[1].text, 'Neighborhood':cols[2].text})

df_nbh = pd.DataFrame(list_codes)

# there are line breaks in the neighborhood column, get rid of them:
df_nbh.Neighborhood = df_nbh.Neighborhood.str.replace('\\n','')

# drop not assigned boroughs
df_nbh = df_nbh.replace({'Not assigned': np.nan})
df_nbh.dropna(subset = ['Borough'],inplace=True)

#reset index:
df_nbh.reset_index(drop=True, inplace = True)
df_nbh.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### group the data by the first 2 columns and combine the neighborhood values into a comma seperated string:

In [3]:
df_nbh = df_nbh.groupby(['PostalCode', 'Borough'], as_index=False).apply(lambda x: ', '.join(map(str,list(x.Neighborhood)))).reset_index()
df_nbh.rename(columns={0:'Neighborhood'}, inplace= True)
df_nbh.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


replace missing neighborhood data with borough:

In [4]:
# replace missing Neighborhood data with Borough:
df_nbh.loc[df_nbh.Neighborhood == 'nan', 'Neighborhood' ] = df_nbh.Borough
df_nbh.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df_nbh.shape

(103, 3)

### Geospatial data
ArcGIS turned out to be the most reliable geocoding service in the geopy library: 

In [6]:
geolocator = ArcGIS(user_agent="toronto_neighborhood")
# initialize location column:
df_nbh['location'] = np.nan
df_nbh['location'] = df_nbh[df_nbh.location.isna()].apply(lambda x : geolocator.geocode( "Toronto, Ontario,  " +  str(x['PostalCode'])), axis=1)
df_nbh['location'].head()

0    (M1B, (43.811525000000074, -79.19551746399998))
1     (M1C, (43.78566500000005, -79.15872457299997))
2     (M1E, (43.76581500000003, -79.17519294699997))
3    (M1G, (43.768369121000035, -79.21758999999997))
4     (M1H, (43.76968799900004, -79.23943999999995))
Name: location, dtype: object

In [7]:
# are there any nans left?
df_nbh['location'].isna().sum()

0

now we need to create the columns for longitude and latitude:


In [8]:
# defining functions to use in apply
def get_longitude(location):
    try:
        return location.longitude
    except Exception:
        return np.nan
def get_latitude(location):
    try:
        return location.latitude
    except Exception:
        return np.nan
    
# use apply to fill the new columns:
df_nbh['Longitude']= df_nbh.location.apply(get_longitude)
df_nbh['Latitude']= df_nbh.location.apply(get_latitude)
df_nbh.head()

,PostalCode,Borough,Neighborhood,location,Longitude,Latitude
0,M1B,Scarborough,"Rouge, Malvern","(M1B, (43.811525000000074, -79.19551746399998))",-79.195517,43.811525
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union","(M1C, (43.78566500000005, -79.15872457299997))",-79.158725,43.785665
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","(M1E, (43.76581500000003, -79.17519294699997))",-79.175193,43.765815
3,M1G,Scarborough,Woburn,"(M1G, (43.768369121000035, -79.21758999999997))",-79.217590,43.768369
4,M1H,Scarborough,Cedarbrae,"(M1H, (43.76968799900004, -79.23943999999995))",-79.239440,43.769688


In [9]:
df_nbh.drop(columns=['location'], inplace=True)
df_nbh.reset_index(drop=True, inplace=True)

In [10]:
df_nbh

,PostalCode,Borough,Neighborhood,Longitude,Latitude
0,M1B,Scarborough,"Rouge, Malvern",-79.195517,43.811525
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",-79.158725,43.785665
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",-79.175193,43.765815
3,M1G,Scarborough,Woburn,-79.217590,43.768369
4,M1H,Scarborough,Cedarbrae,-79.239440,43.769688
5,M1J,Scarborough,Scarborough Village,-79.231750,43.743125
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",-79.263625,43.726276
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",-79.285055,43.713054
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",-79.227925,43.724235
9,M1N,Scarborough,"Birch Cliff, Cliffside West",-79.259967,43.696770
